In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# # Fix for Hopsworks Kafka Producer issue
# import importlib
# import sys

# # Make sure kafka-python is installed
# try:
#     from kafka import KafkaProducer
# except ImportError:
#     raise ImportError("Please install kafka-python: pip install kafka-python")

# # Monkey patch the missing Producer
# import hsfs.core.kafka_producer
# if not hasattr(hsfs.core.kafka_producer, "Producer"):
#     hsfs.core.kafka_producer.Producer = KafkaProducer

In [2]:
HOPSWORKS_PROJECT_NAME = 'msd_taxi_demand'

In [4]:
import os
import sys

# Add the parent directory (project root) to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
# Now try the import
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file
load_dotenv(PARENT_DIR / '.env')
HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [5]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022_01 was already in local storage
File 2022_02 was already in local storage
File 2022_03 was already in local storage
File 2022_04 was already in local storage
File 2022_05 was already in local storage
File 2022_06 was already in local storage
File 2022_07 was already in local storage
File 2022_08 was already in local storage
File 2022_09 was already in local storage
File 2022_10 was already in local storage
File 2022_11 was already in local storage
File 2022_12 was already in local storage
File 2023_01 was already in local storage
File 2023_02 was already in local storage
File 2023_03 was already in local storage
File 2023_04 was already in local storage
File 2023_05 was already in local storage
File 2023_06 was already in local storage
File 2023_07 was already in local storage
File 2023_08 was already in local storage
File 2023_09 was already in local storage
File 2023_10 was already in local storage
File 2023_11 was already in local storage
File 2023_12 was already in local 

In [6]:
print(f'{len(rides)=:,}')

len(rides)=119,133,430


In [7]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:03<00:00, 70.65it/s] 


In [8]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [9]:
import hopsworks

In [10]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

2025-02-27 22:27:45,326 INFO: Initializing external client
2025-02-27 22:27:45,327 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 22:27:47,242 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214654


In [11]:
feature_store = project.get_feature_store()

In [12]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [13]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

In [ ]:
import hsfs
print(hsfs.__version__)

try:
    from confluent_kafka import Producer
    print("Producer class is available")
except ImportError:
    print("Producer class is not available")

In [14]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 6970560/6970560 | Elapsed Time: 06:54 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214654/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)